In [43]:
import os, sys
import json
import re
import time
import pandas as pd
from multiprocessing import Pool
from tqdm import tqdm

import tfidf_searching
from clean_text import cleaned_text

In [44]:
import importlib
importlib.reload(tfidf_searching)

<module 'tfidf_searching' from '/nvme2n1/YangYJworks/@VulApiAlarm/VulLibMiner/tf-idf/tfidf_searching.py'>

In [45]:
with open('/nvme2n1/YangYJworks/@VulApiAlarm/VulLibMiner/myData/train.json', 'r') as f:
    trains = json.load(f)
with open('/nvme2n1/YangYJworks/@VulApiAlarm/VulLibMiner/myData/test.json', 'r') as f:
    tests = json.load(f)

In [46]:
# oskg_folder = '/home/chentianyu/dependency/oskg/'
# node_path = os.path.join(oskg_folder, 'oskg_node2os_20231120163318.csv')
# edge_path = os.path.join(oskg_folder, 'oskg_edges_20231120163318.csv')
# pros_path = os.path.join(oskg_folder, 'oskg_pros_20231120163318.csv')

# nodes = pd.read_csv(node_path, delimiter='\t', header=None)
# edges = pd.read_csv(edge_path, delimiter='\t', header=None)
# pros = pd.read_csv(pros_path, delimiter='\t')

oskg_folder = '/nvme2n1/YangYJworks/@VulApiAlarm/VulLibMiner/myData'
# node_path = os.path.join(oskg_folder, 'oskg_node2os_20231120163318.csv')
# edge_path = os.path.join(oskg_folder, 'oskg_edges_20231120163318.csv')
pros_path = os.path.join(oskg_folder, 'test.csv')

# nodes = pd.read_csv(node_path, delimiter='\t', header=None)
# edges = pd.read_csv(edge_path, delimiter='\t', header=None)
pros = pd.read_csv(pros_path, delimiter='\t')



In [47]:
#获得names列表
pros = pros[pros.apply(lambda x: type(x['name']) == str, axis=1)]
pros_names = set([name.lower() for name in pros.name.to_list() if type(name) == str])

In [48]:
#截取包名
# def get_c_artifact(lib):
#     complete_name = ':'.join(lib.split(':')[1:])
#     artifact = complete_name.split('/')[-1]
#     return artifact.lower()
def get_c_artifact(lib):
    complete_name = ':'.join(lib.split(':')[1:])  # 取第二部分及之后部分
    artifact = complete_name.split('/')[-1]      # 取最后一部分
    artifact = artifact.split(':')[0]            # 去掉版本号
    return artifact.lower()


In [49]:
vulns = trains + tests
vuln_labels = [get_c_artifact(vuln['label']) for vuln in vulns]

In [50]:
vuln_labels

['commons-io', 'spring-core', 'jackson-databind', 'log4j']

In [51]:
name_weight = 4
pros_corpus = pros.drop_duplicates('name')[['name', 'summary']]
pros_corpus.columns = ['object', 'token']
pros_corpus.object = pros_corpus.object.apply(lambda x: x.lower())
pros_corpus.token = pros_corpus.token.apply(lambda x: x if type(x) == str else ' ')
pros_mapping = pros_corpus.set_index('object').to_dict()
pros_corpus.token = pros_corpus.apply(\
                    lambda x: f"{x['object'] * name_weight} {x['token']}", axis=1)

pros_corpus.token = pros_corpus.token.apply(lambda x: cleaned_text(x))
pros_corpus.token = pros_corpus.token.apply(lambda x: ' '.join(x))

In [52]:
pros_corpus

,object,token
0,commons-io,commons iocommons iocommons iocommons io apach...
1,spring-core,spring corespring corespring corespring core c...
2,log4j,log jlog jlog jlog j reliable fast flexible lo...


In [53]:
search_engine = tfidf_searching.TfidfSearching(pros_corpus, 512, 2)

In [54]:
def recall(vuln, search_result, k=128):
    artifact = get_c_artifact(vuln['raw_label'])
    return artifact in search_result[:k]

In [55]:
def fun(vuln):
    search_engine = tfidf_searching.TfidfSearching(pros_corpus, 1024, 2)
    return search_engine.search_topk_objects(cleaned_text(vuln['desc']), [])

In [56]:
with Pool(processes=64) as pool:
    tf_idf_res = list(tqdm(pool.imap(fun, vulns)))


it [00:00, 3892.63it/s]

In [57]:
tf_idf_res

[['commons-io', 'spring-core', 'log4j'],
 ['spring-core', 'commons-io', 'log4j'],
 ['commons-io', 'spring-core', 'log4j'],
 ['log4j', 'commons-io', 'spring-core']]

In [58]:
for vuln, res in zip(vulns, tf_idf_res):
    vuln['top_k'] = [{'lib_name': lib, 'website_description':\
                      pros_mapping['token'][lib]} for lib in res]
    vuln['raw_label'] = vuln['label']
    vuln['labels'] = get_c_artifact(vuln['label'])
    del vuln['label']

In [59]:
vulns

[{'desc': 'A vulnerability in Apache Commons IO allows remote attackers to execute arbitrary code via crafted input.',
  'top_k': [{'lib_name': 'commons-io',
    'website_description': 'Apache Commons IO provides utility classes for IO operations, such as reading and writing files.'},
   {'lib_name': 'spring-core',
    'website_description': 'The core functionality of the Spring Framework, including dependency injection and aspect-oriented programming.'},
   {'lib_name': 'log4j',
    'website_description': 'A reliable, fast, and flexible logging framework (APIs) written in Java.'}],
  'raw_label': 'org.apache.commons:commons-io:2.8.0',
  'labels': 'commons-io'},
 {'desc': 'A deserialization vulnerability in Spring Core allows attackers to execute arbitrary code.',
  'top_k': [{'lib_name': 'spring-core',
    'website_description': 'The core functionality of the Spring Framework, including dependency injection and aspect-oriented programming.'},
   {'lib_name': 'commons-io',
    'website

In [60]:
for idx, vuln in enumerate(vulns):
    vuln['labels'] = [vuln['labels']]

In [61]:
vulns

[{'desc': 'A vulnerability in Apache Commons IO allows remote attackers to execute arbitrary code via crafted input.',
  'top_k': [{'lib_name': 'commons-io',
    'website_description': 'Apache Commons IO provides utility classes for IO operations, such as reading and writing files.'},
   {'lib_name': 'spring-core',
    'website_description': 'The core functionality of the Spring Framework, including dependency injection and aspect-oriented programming.'},
   {'lib_name': 'log4j',
    'website_description': 'A reliable, fast, and flexible logging framework (APIs) written in Java.'}],
  'raw_label': 'org.apache.commons:commons-io:2.8.0',
  'labels': ['commons-io']},
 {'desc': 'A deserialization vulnerability in Spring Core allows attackers to execute arbitrary code.',
  'top_k': [{'lib_name': 'spring-core',
    'website_description': 'The core functionality of the Spring Framework, including dependency injection and aspect-oriented programming.'},
   {'lib_name': 'commons-io',
    'websi

In [62]:
vuln_labels

['commons-io', 'spring-core', 'jackson-databind', 'log4j']

In [63]:
output_dir = '/nvme2n1/YangYJworks/@VulApiAlarm/VulLibMiner/myData/output/'
train_path = os.path.join(output_dir, 'train.json')
valid_path = os.path.join(output_dir, 'valid.json')
test_path = os.path.join(output_dir, 'test.json')

new_train = [vuln for vuln, label in zip(trains, vuln_labels[:-2]) if label in pros_names]
new_test = [vuln for vuln, label in zip(tests, vuln_labels[-2:]) if label in pros_names]

with open(train_path, 'w') as f:
    json.dump(new_train, f)

with open(valid_path, 'w') as f:
    json.dump(new_test, f)

with open(test_path, 'w') as f:
    json.dump(new_test, f)

In [64]:
len(new_test)

1

In [65]:
'ntp' in search_engine.search_topk_objects(cleaned_text(trains[0]['desc']), [])

False